In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# Climate Change Features

In [71]:
cda = pd.read_csv('C:\cygwin\home\Cinthya\Ironhack_bootcamp\Final_Project\data\CDA_ind_na.csv')
cha = pd.read_csv('C:\cygwin\home\Cinthya\Ironhack_bootcamp\Final_Project\data\CHA_ind_na.csv')
fga = pd.read_csv('C:\cygwin\home\Cinthya\Ironhack_bootcamp\Final_Project\data\FGA_ind_na.csv')
nda = pd.read_csv("C:\cygwin\home\Cinthya\Ironhack_bootcamp\Final_Project\data\F_NDA_ind_na.csv")
bca = pd.read_csv('C:\cygwin\home\Cinthya\Ironhack_bootcamp\Final_Project\data\BCA_ind_na.csv')

In [72]:
cda.shape

(220, 29)

# `CDA` column

In [4]:
cda.drop(columns=['code','iso'],inplace=True)
cda_corrected = pd.melt(cda,id_vars =['country'])

In [5]:
cda_corrected['variable'] = cda_corrected['variable'].apply(lambda x: re.sub(r'\bCDA.ind.','',x))
cda_corrected.rename(columns = {'variable':'year','value':'CDA'}, inplace = True)

In [6]:
cda_corrected.isna().sum()

country      0
year         0
CDA        780
dtype: int64

In [7]:
cda_corrected.shape

(5720, 3)

# `CHA` column

In [8]:
cha.shape

(220, 29)

In [9]:
cha.drop(columns=['code','iso'],inplace=True)
cha_corrected = pd.melt(cha,id_vars =['country'])

In [10]:
cha_corrected['variable'] = cha_corrected['variable'].apply(lambda x: re.sub(r'\bCHA.ind.','',x))
cha_corrected.rename(columns = {'variable':'year','value':'CHA'}, inplace = True)

In [11]:
cha_corrected.isna().sum()

country      0
year         0
CHA        780
dtype: int64

In [12]:
cha_corrected.shape

(5720, 3)

# `FGA` column

In [13]:
fga.shape

(220, 29)

In [14]:
fga.drop(columns=['code','iso'],inplace=True)
fga_corrected = pd.melt(fga,id_vars =['country'])

In [15]:
fga_corrected['variable'] = fga_corrected['variable'].apply(lambda x: re.sub(r'\bFGA.ind.','',x))
fga_corrected.rename(columns = {'variable':'year','value':'FGA'}, inplace = True)

In [16]:
fga_corrected.isna().sum()

country       0
year          0
FGA        2600
dtype: int64

In [17]:
fga_corrected.shape

(5720, 3)

# `NDA` column

In [74]:
nda.shape

(220, 29)

In [75]:
nda.drop(columns=['code','iso'],inplace=True)
nda_corrected = pd.melt(nda,id_vars =['country'])

In [76]:
nda_corrected['variable'] = nda_corrected['variable'].apply(lambda x: re.sub(r'\bNDA.ind.','',x))
nda_corrected.rename(columns = {'variable':'year','value':'NDA'}, inplace = True)

In [77]:
nda_corrected.isna().sum()

country      0
year         0
NDA        780
dtype: int64

In [78]:
nda_corrected.shape

(5720, 3)

# `BCA` column

In [18]:
bca.shape

(220, 29)

In [19]:
bca.drop(columns=['code','iso'],inplace=True)
bca_corrected = pd.melt(bca,id_vars =['country'])

In [20]:
bca_corrected['variable'] = bca_corrected['variable'].apply(lambda x: re.sub(r'\bBCA.ind.','',x))
bca_corrected.rename(columns = {'variable':'year','value':'BCA'}, inplace = True)

In [21]:
bca_corrected.isna().sum()

country      0
year         0
BCA        754
dtype: int64

In [22]:
bca_corrected.shape

(5720, 3)

In [23]:
bca_corrected

,country,year,BCA
0,Afghanistan,1995,25.484596
1,Albania,1995,100.000000
2,Algeria,1995,70.996796
3,Andorra,1995,NaN
4,Angola,1995,43.865361
...,...,...,...
5715,Wallis and Futuna Islands,2020,NaN
5716,Western Sahara,2020,NaN
5717,Yemen,2020,100.000000
5718,Zambia,2020,33.143065


# Join all dataset

In [81]:
data1 = pd.merge(cda_corrected,cha_corrected)

In [82]:
data2 = pd.merge(data1,fga_corrected)

In [83]:
data3 = pd.merge(data2,nda_corrected)

In [84]:
data_final = pd.merge(data3,bca_corrected)

In [85]:
countries = list(data_final['country'].unique())

In [86]:
len(countries)

220

# Mising values per country

In [88]:
c2 = [] # CDA
c3 = [] # CHA
c4 = [] # FGA
c5 = [] # NDA
c6 = [] # BCA

for i in countries:
    c2.append(data_final[data_final['country']==str(i)].isna().sum()[2])
    c3.append(data_final[data_final['country']==str(i)].isna().sum()[3])
    c4.append(data_final[data_final['country']==str(i)].isna().sum()[4])
    c5.append(data_final[data_final['country']==str(i)].isna().sum()[5])
    c6.append(data_final[data_final['country']==str(i)].isna().sum()[6])

In [89]:
df = np.array([c2,c3,c4,c5,c6])
data = pd.DataFrame(df)

In [91]:
data['variables'] = ['CDA','CHA','FGA','NDA','BCA']
data.set_index('variables',inplace=True)

In [92]:
data.columns = countries

In [93]:
data = data.T.reset_index()

In [94]:
data.shape

(220, 6)

In [96]:
data['tot'] = data['CDA']+data['CHA']+data['FGA']+data['BCA']

## 26 years for 4 variables

### The countries with more 50% of missing information has to be drop

In [98]:
data['drop'] = data['tot'].apply(lambda x: True if x>=52 else False)

In [99]:
data.groupby('drop').agg({'index':'count'})

,index
drop,
False,188
True,32


In [100]:
drop_data = data[data['drop']==True]
drop_ = list(drop_data['index'])

In [101]:
len(drop_)

32

In [102]:
data_filtered = data[data['drop']==False]
data_filtered.drop(columns=['tot','drop'],inplace=True)
data_filtered.shape

(188, 6)

In [103]:
data_filtered.head(30)

variables,index,CDA,CHA,FGA,NDA,BCA
0,Afghanistan,0,0,26,0,0
1,Albania,0,0,0,0,0
2,Algeria,0,0,0,0,0
4,Angola,0,0,0,0,0
6,Antigua and Barbuda,0,0,0,0,0
7,Argentina,0,0,0,0,0
8,Armenia,0,0,0,0,0
9,Aruba,0,0,26,0,0
10,Australia,0,0,0,0,0
11,Austria,0,0,0,0,0


In [42]:
drop_30 = ['Afghanistan','Aruba','Bahamas','Bangladesh','Barbados','Belize','Benin','Bhutan','Botswana','Burkina Faso','Burundi','Cabo Verde','Cambodia']

In [104]:
data_filtered[30:60]

variables,index,CDA,CHA,FGA,NDA,BCA
34,Cameroon,0,0,0,0,0
35,Canada,0,0,0,0,0
37,Central African Republic,0,0,26,0,0
38,Chad,0,0,26,0,0
39,Chile,0,0,0,0,0
40,China,0,0,0,0,0
41,Colombia,0,0,0,0,0
42,Comoros,0,0,0,0,0
44,Costa Rica,0,0,0,0,0
45,Cote d'Ivoire,0,0,26,0,0


In [44]:
drop_60 = ['Central African Republic','Chad',"Cote d'Ivoire",'Dem. Rep. Congo','Djibouti','Dominica','Dominican Republic','Equatorial Guinea','Eritrea','Eswatini','Fiji']

In [105]:
data_filtered[60:90]

variables,index,CDA,CHA,FGA,NDA,BCA
70,Gabon,0,0,0,0,0
71,Gambia,0,0,26,0,0
72,Georgia,0,0,0,0,0
73,Germany,0,0,0,0,0
74,Ghana,0,0,0,0,0
75,Greece,0,0,0,0,0
77,Grenada,0,0,0,0,0
78,Guatemala,0,0,0,0,0
80,Guinea,0,0,26,0,0
81,Guinea-Bissau,0,0,26,0,0


In [46]:
drop_90 = ['Gambia','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Kiribati']

In [106]:
data_filtered[90:120]

variables,index,CDA,CHA,FGA,NDA,BCA
106,Kyrgyzstan,0,0,0,0,0
107,Laos,0,0,26,0,0
108,Latvia,0,0,0,0,0
109,Lebanon,0,0,26,0,0
110,Lesotho,0,0,26,0,0
111,Liberia,0,0,0,0,0
112,Libya,0,0,0,0,0
114,Lithuania,0,0,0,0,0
115,Luxembourg,0,0,0,0,0
116,Macao,0,0,26,0,0


In [48]:
drop_120 = ['Laos','Lebanon','Lesotho','Macao','Malawi','Maldives','Mali','Marshall Islands','Micronesia','Morocco','Myanmar','Namibia','Nepal']

In [107]:
data_filtered[120:150]

variables,index,CDA,CHA,FGA,NDA,BCA
140,New Zealand,0,0,0,0,0
141,Nicaragua,0,0,26,0,0
142,Niger,0,0,26,0,0
143,Nigeria,0,0,0,0,0
146,North Macedonia,0,0,0,0,0
147,Norway,0,0,0,0,0
148,Oman,0,0,0,0,0
149,Pakistan,0,0,0,0,0
150,Palau,0,0,26,0,0
151,Panama,0,0,26,0,0


In [50]:
drop_150 = ['Nicaragua','Niger','Palau','Panama','Paraguay','Qatar','Samoa','San Marino','Sao Tome and Principe','Senegal']

In [108]:
data_filtered[150:180]

variables,index,CDA,CHA,FGA,NDA,BCA
173,Seychelles,0,0,26,0,0
174,Sierra Leone,0,0,0,0,0
175,Singapore,0,0,0,0,0
177,Slovakia,0,0,0,0,0
178,Slovenia,0,0,0,0,0
179,Solomon Islands,0,0,26,0,0
181,South Africa,0,0,0,0,0
182,South Korea,0,0,0,0,0
184,Spain,0,0,0,0,0
185,Sri Lanka,0,0,26,0,0


In [52]:
drop_180 = ['Seychelles','Solomon Islands','Sri Lanka','Sudan','Tanzania','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia','Uganda']

In [109]:
data_filtered[180:]

variables,index,CDA,CHA,FGA,NDA,BCA
210,Uruguay,0,0,0,0,0
211,Uzbekistan,0,0,0,0,0
212,Vanuatu,0,0,26,0,0
213,Venezuela,0,0,0,0,0
214,Viet Nam,0,0,26,0,0
217,Yemen,0,0,26,0,0
218,Zambia,0,0,0,0,0
219,Zimbabwe,0,0,26,0,0


In [110]:
drop_200 = ['Vanuatu','Viet Nam','Yemen','Zimbabwe']

In [111]:
final_drop = drop_ + drop_30 + drop_60 + drop_90 + drop_120 + drop_150 + drop_180 + drop_200

In [112]:
len(final_drop)

101

In [113]:
data_final.shape

(5720, 7)

In [114]:
for country in final_drop:
    data_final = data_final[data_final.country != str(country)]

In [115]:
data_final.shape

(3094, 7)

In [116]:
data_final.isna().sum()

country    0
year       0
CDA        0
CHA        0
FGA        0
NDA        0
BCA        0
dtype: int64

In [117]:
data_final.describe()

,CDA,CHA,FGA,NDA,BCA
count,3094.000000,3094.000000,3094.000000,3094.000000,3094.000000
mean,45.760713,69.972617,77.019395,60.496138,59.863459
std,20.497251,28.330927,26.156018,28.685344,31.585344
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.922921,51.522771,70.620648,40.066229,35.641234
50%,45.938993,75.715124,87.696875,63.143266,60.073786
75%,58.473675,100.000000,92.543430,81.889906,94.171052
max,100.000000,100.000000,100.000000,100.000000,100.000000


In [118]:
data_final

,country,year,CDA,CHA,FGA,NDA,BCA
1,Albania,1995,100.000000,73.072520,0.000000,78.221486,100.000000
2,Algeria,1995,40.836434,98.636534,96.334651,72.620891,70.996796
4,Angola,1995,12.888007,70.729875,0.000000,100.000000,43.865361
6,Antigua and Barbuda,1995,44.483927,80.575637,0.000000,36.690524,68.683208
7,Argentina,1995,40.485271,100.000000,100.000000,63.435774,45.942816
...,...,...,...,...,...,...,...
5709,United States of America,2020,66.956590,76.716102,94.188345,78.133196,100.000000
5710,Uruguay,2020,48.710428,87.313678,73.872069,73.410235,3.282750
5711,Uzbekistan,2020,67.872867,43.951256,78.654405,21.178605,97.211440
5713,Venezuela,2020,57.814027,83.278582,95.878376,68.376079,61.272558


# Export to csv and excel

In [119]:
data_final.to_excel('data_2020.xlsx')
data_final.to_csv('data_2020.csv')